In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

arquivo_todos_csv = spark.read.option('delimiter', ',').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedCsv.csv/part-0000[0-3]-11b1c6d7-1ac5-4469-bdaa-adb68ed234ae-c000.csv')
arquivo_todos_json = spark.read.option('delimiter', ',').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedJson.csv/part-0000[0-4]-bc735acd-498d-4217-8f2f-29d5dbf13433-c000.csv')
arquivo_todos_txt = spark.read.option('delimiter', ',').option('header', 'true').csv('s3a://stc-grupo04-henrique-sptech-data-transform/preTratedTxt.csv/part-00000-13f5c5fb-d0de-412a-9471-1595218a4ff6-c000.csv')
print('arquivos lidos')

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1fd299f6-ef28-4489-a3e1-facf7c7626bb;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 414ms :: artifacts dl 13ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

arquivos lidos


In [2]:
print(arquivo_todos_csv.count())
print(arquivo_todos_json.count())
print(arquivo_todos_txt.count())

1000000


1000001
28


In [3]:
arquivo_todos_csv.printSchema()
arquivo_todos_json.printSchema()
arquivo_todos_txt.printSchema()

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- ETNIA: string (nullable = true)
 |-- EVOLUCAO_CASO: string (nullable = true)
 |-- PRIMEIRA_DOSE: string (nullable = true)
 |-- SEGUNDA_DOSE: string (nullable = true)

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- ASMA: string (nullable = true)
 |-- CARDIOPATIA: string (nullable = true)
 |-- DIABETE: string (nullable = true)
 |-- OBESIDADE: string (nullable = true)
 |-- SIN_DOWN: string (nullable = true)

root
 |-- TERRITORIALIDADES: string (nullable = true)
 |-- MEDIA_ANOS_ESTUDOS: string (nullable = true)
 |-- INDICE_EDUCACAO: string (nullable = true)
 |-- LONGEVIDADE: string (nullable = true)
 |-- RENDA: string (nullable = true)
 |-- IDH: string (nullable = true)



In [8]:
#limpar registros em branco (se tiver mesmo)
#arquivo_semi_csv=arquivo_todos_csv.filter(col("ESTADO") != "")

from pyspark.sql.functions import concat


arquivo_semi_csv_1=(arquivo_todos_csv.select(
    col('ESTADO').alias('ESTADO_csv'),
    col('SEXO').alias('SEXO_csv'),
    col('IDADE').cast(IntegerType()).alias('IDADE_csv'),
    col('ETNIA'),
    col('EVOLUCAO_CASO'),
    col('PRIMEIRA_DOSE'),
    col('SEGUNDA_DOSE')
)).filter(col("ESTADO") != "").filter(col("SEXO")!= "").filter(col("IDADE")!= "") #verificação essencial 
#.filter(col("ETNIA")!= "").filter(col("EVOLUCAO_CASO")!= "")   #verificação opcionais(personalizar tabela)




arquivo_semi_json=arquivo_todos_json.filter(col("ESTADO") != "").filter(col("SEXO")!= "").filter(col("IDADE")!= "") #verificação essencial
.filter(concat(col('ASMA'),col('CARDIOPATIA'),col('DIABETE'),col('OBESIDADE'),col('SIN_DOWN')).like("%SIM%"))       #verificação opcional

print(arquivo_semi_csv_1.count())
print(arquivo_semi_json.count())

970888


66730


In [10]:
from pyspark.sql.functions import upper 

join_csv_json = arquivo_semi_json.join(arquivo_semi_csv_1,((arquivo_semi_json.ESTADO==arquivo_semi_csv_1.ESTADO_csv) & 
                                                          (arquivo_semi_json.SEXO==upper(arquivo_semi_csv_1.SEXO_csv)) &
                                                          (arquivo_semi_json.IDADE==arquivo_semi_csv_1.IDADE_csv)
                                                          & (arquivo_semi_json.ESTADO=="SP")  #Opcional (Personalizavel)
                                                          ),"inner")



In [11]:
join_csv_json.printSchema()

root
 |-- ESTADO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- IDADE: string (nullable = true)
 |-- ASMA: string (nullable = true)
 |-- CARDIOPATIA: string (nullable = true)
 |-- DIABETE: string (nullable = true)
 |-- OBESIDADE: string (nullable = true)
 |-- SIN_DOWN: string (nullable = true)
 |-- ESTADO_csv: string (nullable = true)
 |-- SEXO_csv: string (nullable = true)
 |-- IDADE_csv: integer (nullable = true)
 |-- ETNIA: string (nullable = true)
 |-- EVOLUCAO_CASO: string (nullable = true)
 |-- PRIMEIRA_DOSE: string (nullable = true)
 |-- SEGUNDA_DOSE: string (nullable = true)



In [12]:
#limpando coluna ESTADO SEXO IDADE duplicada
join_csv_json_final = join_csv_json.select(
    col('ESTADO'),
    col('SEXO'),
    col('IDADE'),
    col('ETNIA'),
    col('ASMA'),
    col('CARDIOPATIA'),
    col('DIABETE'),
    col('SIN_DOWN'),
    col('EVOLUCAO_CASO'),
    col('PRIMEIRA_DOSE'),
    col('SEGUNDA_DOSE')
).limit(1000000)

In [13]:
type(join_csv_json_final)
print(join_csv_json_final.count())

1000000


In [14]:
#Salvando S3
join_csv_json_final.write.mode('overwrite').option("header", "true").csv('s3a://stc-grupo04-henrique-sptech-data-final/trated1.csv')

22/05/31 01:13:54 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/05/31 01:14:03 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [17]:
#Salvando Azure
print("inserindo azure")
join_csv_json_final.write \
.format('com.microsoft.sqlserver.jdbc.spark') \
.mode('overwrite') \
.option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
.option('url', 'jdbc:sqlserver://servidor-grupo04.database.windows.net;databaseName=grupo04;') \
.option('dbtable', 'sp_join_csv_json') \
.option('user', 'urubu100') \
.option('password', 'Urubu1@@') \
.save()
print("Salvo")

inserindo azure


Salvo
